In [1]:
# Import packages
import numpy as np
import pandas as pd

# Exploratory Analysis

Before we begin fitting models, let's explore our data set a bit first
1. Event count for each patient. This calculates the number of diagnosis, drugs taken, and lab test of living patients and dead patients.
2. Encounter count calculates how many visits a patient had to the ICU.
3. Record length records how long the patient has been tracked in our dataset, the difference between the last day recorded and first day recorded for a patient

## Event count

In [2]:
events = pd.read_csv("../data/train/events.csv")
mortality = pd.read_csv("../data/train/mortality_events.csv")

morts = mortality['patient_id']
dead = events[events['patient_id'].isin(morts)]
alive = events[~events['patient_id'].isin(morts)]
display(dead.describe(), events.describe(), alive.describe())


,patient_id,value
count,491007.000000,4.442380e+05
mean,10050.837397,7.205522e+01
std,6091.513865,2.575690e+03
min,12.000000,-4.700000e+01
25%,5072.000000,3.010000e+00
50%,9967.000000,1.500000e+01
75%,15038.000000,4.700000e+01
max,25075.000000,1.115000e+06


,patient_id,value
count,740066.000000,6.675170e+05
mean,9415.493002,7.050876e+01
std,5808.988279,2.171407e+03
min,12.000000,-4.700000e+01
25%,4683.000000,3.110000e+00
50%,8904.000000,1.490000e+01
75%,14180.000000,4.600000e+01
max,25075.000000,1.115000e+06


,patient_id,value
count,249059.000000,223279.000000
mean,8162.944226,67.431897
std,4975.076120,946.895486
min,80.000000,-32.000000
25%,3917.000000,3.300000
50%,7299.000000,14.400000
75%,12171.000000,44.000000
max,18480.000000,117500.000000


In [3]:
alive_group = alive.groupby(['patient_id']).size().reset_index(name='count')
dead_group = dead.groupby(['patient_id']).size().reset_index(name='count')

display(alive_group.head())
display(dead_group.head())

,patient_id,count
0,80,185
1,99,238
2,197,332
3,198,944
4,306,233


,patient_id,count
0,12,868
1,19,177
2,41,1092
3,106,622
4,112,556


## Encounter count

In [4]:
alive_group = alive.groupby(['patient_id', 'timestamp']).size().reset_index(name='count')
dead_group = dead.groupby(['patient_id', 'timestamp']).size().reset_index(name='count')

alive_encounter_count = alive_group.groupby(['patient_id']).agg('count').reset_index()
alive_encounter_count.drop('timestamp', axis=1, inplace=True)
dead_encounter_count = dead_group.groupby(['patient_id']).agg('count').reset_index()
dead_encounter_count.drop('timestamp', axis=1, inplace=True)

alive_encounter_count.head()

,patient_id,count
0,80,5
1,99,10
2,197,9
3,198,22
4,306,7


## Record length

In [5]:
alive['timestamp'] = pd.to_datetime(alive['timestamp'])
a_g1 = alive.groupby(['patient_id'])
a_g2 = a_g1.agg(lambda x: x['timestamp'].max() - x['timestamp'].min()).reset_index()
a_g2.head()

/Users/samuelyusuf/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,patient_id,event_id,event_description,timestamp,value
0,80,8 days,8 days,8 days,8 days
1,99,1267 days,1267 days,1267 days,1267 days
2,197,9 days,9 days,9 days,9 days
3,198,300 days,300 days,300 days,300 days
4,306,6 days,6 days,6 days,6 days


In [6]:
alive_record_length = a_g2[['patient_id', 'value']]

In [7]:
alive_record_length.head()

,patient_id,value
0,80,8 days
1,99,1267 days
2,197,9 days
3,198,300 days
4,306,6 days


In [8]:
dead.head()

,patient_id,event_id,event_description,timestamp,value
0,12,DIAG440649,Primary malignant neoplasm of head of pancreas,2011-12-06,1.0
1,12,DIAG201070,Cholelithiasis AND cholecystitis without obstr...,2011-12-06,1.0
2,12,DIAG321462,Cardiac complication,2011-12-06,1.0
3,12,DIAG321042,Cardiac arrest,2011-12-06,1.0
4,12,DIAG435141,Hemorrhage AND/OR hematoma complicating procedure,2011-12-06,1.0


In [9]:
dead['timestamp'] = pd.to_datetime(dead['timestamp'])
m_g1 = dead.groupby(['patient_id'])
m_g2 = m_g1.agg(lambda x: x['timestamp'].max() - x['timestamp'].min()).reset_index()
m_g2['value'] = m_g2['value'].dt.days
dead_record_length = m_g2[['patient_id', 'value']]
dead_record_length.head()

/Users/samuelyusuf/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,patient_id,value
0,12,15
1,19,4
2,41,43
3,106,6
4,112,839


## Calculate index date

In [10]:
mortality.head()

,patient_id,timestamp,label
0,19,2014-03-04,1
1,12,2011-12-19,1
2,41,2014-02-15,1
3,106,2015-08-11,1
4,112,2011-04-23,1


In [11]:
morts = mortality['patient_id']

alive = events[~events.patient_id.isin(morts)]

In [12]:
mortality['timestamp'] = pd.to_datetime(mortality['timestamp']).dt.date
mortality.head()

,patient_id,timestamp,label
0,19,2014-03-04,1
1,12,2011-12-19,1
2,41,2014-02-15,1
3,106,2015-08-11,1
4,112,2011-04-23,1


In [13]:
mortality['indx_date'] = mortality['timestamp'] - pd.to_timedelta(30, unit='d')
mortality.head()
dead = mortality[['patient_id', 'indx_date']]
dead.head()

,patient_id,indx_date
0,19,2014-02-02
1,12,2011-11-19
2,41,2014-01-16
3,106,2015-07-12
4,112,2011-03-24


In [14]:
a_g1 = alive.groupby(['patient_id']).agg({'timestamp': {'indx_date': 'max'}}).reset_index()
a_g1.columns = ['patient_id', 'indx_date']
a_g1.tail()

/Users/samuelyusuf/anaconda2/lib/python2.7/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


,patient_id,indx_date
495,18326,2010-11-22
496,18367,2013-08-04
497,18452,2014-06-20
498,18455,2012-10-24
499,18480,2012-12-22


In [15]:
indx_date = dead.append(a_g1)
indx_date.describe()
indx_date[indx_date['patient_id'] == 24581]

,patient_id,indx_date
49,24581,2013-11-07


## Filter event

In [16]:
display(events.head())
display(indx_date.head())

evts = pd.merge(events, indx_date, on='patient_id')
evts.head()

,patient_id,event_id,event_description,timestamp,value
0,12,DIAG440649,Primary malignant neoplasm of head of pancreas,2011-12-06,1.0
1,12,DIAG201070,Cholelithiasis AND cholecystitis without obstr...,2011-12-06,1.0
2,12,DIAG321462,Cardiac complication,2011-12-06,1.0
3,12,DIAG321042,Cardiac arrest,2011-12-06,1.0
4,12,DIAG435141,Hemorrhage AND/OR hematoma complicating procedure,2011-12-06,1.0


,patient_id,indx_date
0,19,2014-02-02
1,12,2011-11-19
2,41,2014-01-16
3,106,2015-07-12
4,112,2011-03-24


,patient_id,event_id,event_description,timestamp,value,indx_date
0,12,DIAG440649,Primary malignant neoplasm of head of pancreas,2011-12-06,1.0,2011-11-19
1,12,DIAG201070,Cholelithiasis AND cholecystitis without obstr...,2011-12-06,1.0,2011-11-19
2,12,DIAG321462,Cardiac complication,2011-12-06,1.0,2011-11-19
3,12,DIAG321042,Cardiac arrest,2011-12-06,1.0,2011-11-19
4,12,DIAG435141,Hemorrhage AND/OR hematoma complicating procedure,2011-12-06,1.0,2011-11-19


In [17]:
evts['timestamp'] = pd.to_datetime(evts['timestamp']).dt.date
evts['indx_date'] = pd.to_datetime(evts['indx_date']).dt.date
filt = evts[(evts['timestamp'] <= evts['indx_date']) & (evts['timestamp'] >= evts['indx_date'] - pd.to_timedelta(2000, unit='d'))]
filt.head()

,patient_id,event_id,event_description,timestamp,value,indx_date
868,19,DIAG433329,Closed fracture of second cervical vertebra,2013-02-19,1.0,2014-02-02
869,19,DIAG81902,Urinary tract infectious disease,2013-02-19,1.0,2014-02-02
870,19,DIAG74082,Acontractile detrusor,2013-02-19,1.0,2014-02-02
871,19,DIAG436539,Closed fracture of patella,2013-02-19,1.0,2014-02-02
872,19,DIAG80502,Osteoporosis,2013-02-19,1.0,2014-02-02


In [18]:
display(filt[(filt['patient_id'] == 2293) & (filt['event_id'] == 'LAB3019550')])
filt.info()

,patient_id,event_id,event_description,timestamp,value,indx_date
106962,2293,LAB3019550,Sodium [Moles/volume] in Serum or Plasma,2007-11-02,140.0,2010-06-02
107034,2293,LAB3019550,Sodium [Moles/volume] in Serum or Plasma,2007-11-04,139.0,2010-06-02
107065,2293,LAB3019550,Sodium [Moles/volume] in Serum or Plasma,2007-11-04,139.0,2010-06-02
107105,2293,LAB3019550,Sodium [Moles/volume] in Serum or Plasma,2007-11-05,138.0,2010-06-02
107136,2293,LAB3019550,Sodium [Moles/volume] in Serum or Plasma,2007-11-06,140.0,2010-06-02
107162,2293,LAB3019550,Sodium [Moles/volume] in Serum or Plasma,2007-11-07,143.0,2010-06-02
107172,2293,LAB3019550,Sodium [Moles/volume] in Serum or Plasma,2007-11-08,143.0,2010-06-02
107215,2293,LAB3019550,Sodium [Moles/volume] in Serum or Plasma,2007-11-09,141.0,2010-06-02
107247,2293,LAB3019550,Sodium [Moles/volume] in Serum or Plasma,2007-11-17,141.0,2010-06-02
107322,2293,LAB3019550,Sodium [Moles/volume] in Serum or Plasma,2007-11-18,142.0,2010-06-02


<class 'pandas.core.frame.DataFrame'>
Int64Index: 556347 entries, 868 to 740064
Data columns (total 6 columns):
patient_id           556347 non-null int64
event_id             556347 non-null object
event_description    556347 non-null object
timestamp            556347 non-null object
value                500684 non-null float64
indx_date            556347 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 29.7+ MB


## Aggregate events

In [19]:
feature_map = pd.read_csv("../data/train/event_feature_map.csv")
feature_map.head()

,idx,event_id
0,1,DIAG132397
1,2,DIAG132408
2,3,DIAG132446
3,4,DIAG132583
4,5,DIAG132643


In [20]:
agg = pd.merge(filt, feature_map, on='event_id')
agg.dropna(inplace=True)
agg.head()

,patient_id,event_id,event_description,timestamp,value,indx_date,idx
0,19,DIAG433329,Closed fracture of second cervical vertebra,2013-02-19,1.0,2014-02-02,1202
1,19,DIAG81902,Urinary tract infectious disease,2013-02-19,1.0,2014-02-02,2028
2,3009,DIAG81902,Urinary tract infectious disease,2013-12-26,1.0,2015-04-09,2028
3,6006,DIAG81902,Urinary tract infectious disease,2013-04-20,1.0,2014-04-24,2028
4,157,DIAG81902,Urinary tract infectious disease,2013-01-18,1.0,2013-02-26,2028


In [21]:
agg1 = agg.groupby(['patient_id', 'event_id', 'idx']).agg({'value': 'count'}).reset_index()
agg1.head()
feature_pairs = agg1[['patient_id','idx', 'value']]
feature_pairs.rename(columns={'idx': 'feature_id', 'value': 'feature_value'}, inplace=True)

/Users/samuelyusuf/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [22]:
feature_pairs.head()

,patient_id,feature_id,feature_value
0,19,719,1
1,19,744,1
2,19,784,1
3,19,786,1
4,19,1202,1


In [23]:
feature_pairs['max_feature_value'] = feature_pairs.groupby(['feature_id'])['feature_value'].transform(np.max)

In [24]:
feature_pairs['min_feature_value'] = feature_pairs.groupby(['feature_id'])['feature_value'].transform(np.min)

In [25]:
feature_pairs.head(10)

,patient_id,feature_id,feature_value,max_feature_value,min_feature_value
0,19,719,1,4,1
1,19,744,1,1,1
2,19,784,1,3,1
3,19,786,1,3,1
4,19,1202,1,1,1
5,19,1411,1,1,1
6,19,1895,1,1,1
7,19,2001,1,2,1
8,19,2028,1,2,1
9,19,2040,2,22,1


In [26]:
feature_pairs['feature_value'] = (feature_pairs['feature_value']) / (feature_pairs['max_feature_value'])

In [27]:
feature_pairs.head(20)

,patient_id,feature_id,feature_value,max_feature_value,min_feature_value
0,19,719,0.250000,4,1
1,19,744,1.000000,1,1
2,19,784,0.333333,3,1
3,19,786,0.333333,3,1
4,19,1202,1.000000,1,1
5,19,1411,1.000000,1,1
6,19,1895,1.000000,1,1
7,19,2001,0.500000,2,1
8,19,2028,0.500000,2,1
9,19,2040,0.090909,22,1


In [28]:
feature_pairs.describe()

,patient_id,feature_id,feature_value,max_feature_value,min_feature_value
count,64209.000000,64209.000000,64209.000000,64209.000000,64209.000000
mean,9397.951424,2528.773287,0.200134,133.299771,1.003847
std,5709.579119,668.648141,0.282694,161.729717,0.117869
min,19.000000,1.000000,0.001898,1.000000,1.000000
25%,4692.000000,2432.000000,0.026634,9.000000,1.000000
50%,8904.000000,2766.000000,0.077482,55.000000,1.000000
75%,13653.000000,2914.000000,0.244755,188.000000,1.000000
max,25075.000000,3179.000000,1.000000,527.000000,14.000000


In [29]:
feature_pairs['feature_value'].max(), feature_pairs['feature_value'].min()

(1.0, 0.0018975332068311196)

In [30]:
feature_pairs.describe()
#feature_pairs.head()

,patient_id,feature_id,feature_value,max_feature_value,min_feature_value
count,64209.000000,64209.000000,64209.000000,64209.000000,64209.000000
mean,9397.951424,2528.773287,0.200134,133.299771,1.003847
std,5709.579119,668.648141,0.282694,161.729717,0.117869
min,19.000000,1.000000,0.001898,1.000000,1.000000
25%,4692.000000,2432.000000,0.026634,9.000000,1.000000
50%,8904.000000,2766.000000,0.077482,55.000000,1.000000
75%,13653.000000,2914.000000,0.244755,188.000000,1.000000
max,25075.000000,3179.000000,1.000000,527.000000,14.000000


## Create features

In [32]:
patient_features = {}
for patient in set(feature_pairs['patient_id']):
    for index, row in feature_pairs[feature_pairs['patient_id'] == patient].iterrows():
        if patient in patient_features.keys():
            patient_features[patient].append((row['feature_id'], row['feature_value']))
        else:
            patient_features[patient] = [(row['feature_id'], row['feature_value'])]

In [34]:
mortality.head()

,patient_id,timestamp,label,indx_date
0,19,2014-03-04,1,2014-02-02
1,12,2011-12-19,1,2011-11-19
2,41,2014-02-15,1,2014-01-16
3,106,2015-08-11,1,2015-07-12
4,112,2011-04-23,1,2011-03-24


In [35]:
print(len(mortality))
len(set(mortality['patient_id']))

500


500

In [36]:
mortality_dict = {}
for index, row in mortality.iterrows():
    mortality_dict[row['patient_id']] = row['label']

## Save svmlight

In [38]:
aggregated_events = patient_features.copy()
features = aggregated_events.copy()

In [64]:
len(features)*0.25

209.0

In [40]:
for key in aggregated_events.keys():
    if key in mortality_dict.keys():
        features[key].insert(0,1)
    else:
        features[key].insert(0,0)

In [42]:
deliverable1 = open('../deliverables/features_svmlight_test.train', 'wb')
deliverable2 = open('../deliverables/features_test.train', 'wb')

In [44]:
for key, values in sorted(features.items()):
    first_part = str(values[0]) + " "
    sec_part = " ".join(str(int(val[0])) + ":" + format(val[1], '.6f') for val in sorted(features[key][1:])) 
    deliverable1.write(first_part + sec_part + '\n')
    deliverable2.write(str(int(key)) + " " + first_part + sec_part + '\n')
    

# Predictive Modeling!

Now that the data's clean and we have our features in sparse vector representation, let's try our data set on a few classifiers

In [45]:
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
import utils

## Logistic Regression

In [46]:
X_train, Y_train = utils.get_data_from_svmlight("../deliverables/features_svmlight.train")

In [47]:
X_train[1]

<1x3190 sparse matrix of type '<type 'numpy.float64'>'
	with 128 stored elements in Compressed Sparse Row format>

In [48]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)

Y_pred_lr = logreg.predict(X_train)

## SVM

In [50]:
svm = LinearSVC()
svm.fit(X_train, Y_train)

Y_pred_svm = svm.predict(X_train)

## Decision Tree

In [52]:
tree = DecisionTreeClassifier(max_depth=5)
tree.fit(X_train, Y_train)

Y_pred_tree = tree.predict(X_train)

## Classification Metrics

In [54]:
def classification_metrics(Y_pred, Y_true):
    #NOTE: It is important to provide the output in the same order
    acc = accuracy_score(Y_true, Y_pred)
    auc = roc_auc_score(Y_true, Y_pred)
    prec = precision_score(Y_true, Y_pred)
    rec = recall_score(Y_true, Y_pred)
    f1score = f1_score(Y_true, Y_pred)

    return acc, auc, prec, rec, f1score

In [55]:
#input: Name of classifier, predicted labels, actual labels
def display_metrics(classifierName,Y_pred,Y_true):
    print "______________________________________________"
    print "Classifier: "+classifierName
    acc, auc_, precision, recall, f1score = classification_metrics(Y_pred,Y_true)
    print "Accuracy: "+str(acc)
    print "AUC: "+str(auc_)
    print "Precision: "+str(precision)
    print "Recall: "+str(recall)
    print "F1-score: "+str(f1score)
    print "______________________________________________"
    print ""

In [56]:
display_metrics("Logistic Regression",Y_pred_lr,Y_train)
display_metrics("SVM",Y_pred_svm,Y_train)
display_metrics("Decision Tree",Y_pred_tree,Y_train)

______________________________________________
Classifier: Logistic Regression
Accuracy: 0.954545454545
AUC: 0.945404761905
Precision: 0.986928104575
Recall: 0.89880952381
F1-score: 0.940809968847
______________________________________________

______________________________________________
Classifier: SVM
Accuracy: 0.994019138756
AUC: 0.994511904762
Precision: 0.988200589971
Recall: 0.997023809524
F1-score: 0.992592592593
______________________________________________

______________________________________________
Classifier: Decision Tree
Accuracy: 0.776315789474
AUC: 0.747595238095
Precision: 0.792156862745
Recall: 0.60119047619
F1-score: 0.68358714044
______________________________________________



Classifiers performs very well on the training set. Let's see their performance the test data set.

In [57]:
X_test, Y_test = utils.get_data_from_svmlight("../data/features_svmlight.validate")

In [58]:
# Logistic Regression
Y_pred_lr = logreg.predict(X_test)
# SVM
Y_pred_svm = svm.predict(X_test)
# Decision tree
Y_pred_tree = tree.predict(X_test)

In [59]:
display_metrics("Logistic Regression",Y_pred_lr,Y_test)
display_metrics("SVM",Y_pred_svm,Y_test)
display_metrics("Decision Tree",Y_pred_tree,Y_test)

______________________________________________
Classifier: Logistic Regression
Accuracy: 0.738095238095
AUC: 0.7375
Precision: 0.680412371134
Recall: 0.733333333333
F1-score: 0.705882352941
______________________________________________

______________________________________________
Classifier: SVM
Accuracy: 0.738095238095
AUC: 0.738888888889
Precision: 0.676767676768
Recall: 0.744444444444
F1-score: 0.708994708995
______________________________________________

______________________________________________
Classifier: Decision Tree
Accuracy: 0.671428571429
AUC: 0.656944444444
Precision: 0.632911392405
Recall: 0.555555555556
F1-score: 0.591715976331
______________________________________________



Our classifiers seem to perform pretty well on unknown data, on our test data set with accuracy of about 73%.

It seems a bit old that the Logistic Regression classifier and the SVM classifier had the same accuracy score. Let's take a look at the prediction labels for both classifiers to see if they actually gave the same labels to every patient.

In [60]:
print("Logistic Regression Y_pred")
display(Y_pred_lr)
print("SVM Y_pred")
display(Y_pred_svm)

Logistic Regression Y_pred


array([ 0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,
        1.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,
        0.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,
        0.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,
        1.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,
        1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,
        0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  0.,  1.,  0.,  1.,
        0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
        0.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,  0.,  0.,  1.,  0

SVM Y_pred


array([ 0.,  1.,  0.,  1.,  1.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,
        0.,  0.,  0.,  1.,  1.,  0.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,
        1.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,
        0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,
        0.,  1.,  1.,  0.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,
        0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
        0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,
        0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  0.,  1.,  0.,  1.,
        0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,
        0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  0

It appears that both classifiers gave the same number of positives and thus ended up with the same accuracy score. However, they did not assign the same labels to every patient.

Next, we should explore using Cross fold validation to evaluate and improve our predictive model

## K-fold cross validation

In [61]:
from sklearn.cross_validation import KFold, ShuffleSplit
from numpy import mean

/Users/samuelyusuf/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [67]:
X,Y = utils.get_data_from_svmlight("../deliverables/features_svmlight.train")
kf = KFold(X.shape[0], n_folds=5, random_state= 545510477)

accuracy_scores = []
auc_scores = []

for train_index, test_index in kf:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)

    y_pred = logreg.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    
    accuracy_scores.append(acc)
    auc_scores.append(auc)

print("KFold mean accuracy: ", np.mean(accuracy_scores), "KFold mean AUC: ", np.mean(auc_scores))

('KFold mean accuracy: ', 0.72132164242942687, 'KFold mean AUC: ', 0.70757733030284675)


In [68]:
accuracy_scores
auc_scores

[0.68611111111111112,
 0.69493608652900685,
 0.74677419354838714,
 0.7236559139784946,
 0.68640934634723449]

In [69]:
## KFold for SVM
accuracy_scores = []
auc_scores = []

for train_index, test_index in kf:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    svm = LinearSVC()
    svm.fit(X_train, y_train)

    y_pred = svm.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    
    accuracy_scores.append(acc)
    auc_scores.append(auc)

print("KFold mean accuracy: ", np.mean(accuracy_scores), "KFold mean AUC: ", np.mean(auc_scores))

('KFold mean accuracy: ', 0.71298118049615056, 'KFold mean AUC: ', 0.71111108854860894)


## Shuffle split cross validation

In [ ]:
ss = ShuffleSplit(X.shape[0], n_iter=5, test_size=0.25, random_state=545510477)

accuracy_scores = []
auc_scores = []

for train_index, test_index in ss:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)

    y_pred = logreg.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    
    accuracy_scores.append(acc)
    auc_scores.append(auc)

print("ShuffleSplit mean accuracy: ", np.mean(accuracy_scores), "ShuffleSplit mean AUC: ", np.mean(auc_scores))

## Random Forest Classifier

In [65]:
from sklearn.ensemble import RandomForestClassifier
import etl
import models_partc

In [66]:
def my_features():
	#TODO: complete this
    
    ## Creat pandas df from csv
    events, mortality, feature_map = etl.read_csv('../data/train/')
    
    
    patient_features, mortality = etl.create_features(events, mortality, feature_map)
    
    features = patient_features.copy()

    
    feature_length = len(features)
    
    # Training set = 75% of patients, Test set = 25% of patients
    features_train = dict(features.items()[(feature_length/4):])
    features_test = dict(features.items()[:(feature_length/4)])
    
    etl.save_svmlight(features_train, mortality, "../data/train_features_svmlight.train", "../data/train_features.train")
    etl.save_svmlight(features_test, mortality, "../data/test_features_svmlight.train", "../deliverables/test_features.txt")

        
    X_train, Y_train = utils.get_data_from_svmlight("../data/train_features_svmlight.train")
    X_test, Y_test = utils.get_data_from_svmlight("../data/test_features_svmlight.train")
    
    return X_train, Y_train, X_test